In [25]:
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.keys import Keys

# Credenciales de Twitter
username = 'ScrapeoUPM'
password = 'scrapeafutbol'
email = 'oscar.marin@alumnos.upm.es'

SCROLL_PAUSE_TIME = 2  # Tiempo de espera para que carguen nuevos tweets

# Iniciar el navegador con Selenium y WebDriver Manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Navegar a la página de inicio de sesión de Twitter
driver.get('https://twitter.com/login')
time.sleep(5)  # Esperar a que cargue la página

# Buscar el campo del nombre de usuario e ingresar el nombre de usuario o correo
username_input = driver.find_element(By.NAME, 'text')
username_input.send_keys(username)
username_input.send_keys(Keys.RETURN)
# driver.find_element(By.XPATH, '//*[@id="layers"]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/button[2]').click()
time.sleep(2)

# Buscar el campo de confirmación de correo
email_input = driver.find_element(By.NAME, 'text')
email_input.send_keys(email)
# driver.find_element(By.XPATH, '//*[@id="react-root"]/div/div/div/main/div/div/div/div[2]/div[2]/div[2]/div/div/div/div/button').click()
email_input.send_keys(Keys.RETURN)
time.sleep(2)
#Presionar tecla enter

# Buscar el campo de contraseña e ingresar la contraseña
password_input = driver.find_element(By.NAME, 'password')
password_input.send_keys(password)

# Hacer clic en el botón de iniciar sesión
# driver.find_element(By.XPATH, '//*[@id="layers"]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/button').click()
password_input.send_keys(Keys.RETURN)
time.sleep(5)  # Esperar unos segundos para asegurarse de que el login se ha realizado

# Ahora que has iniciado sesión, puedes navegar a Twitter y hacer scraping
driver.get('https://twitter.com/search?q=Ferran Torres')

# Esperar que los tweets carguen
time.sleep(5)

# Extraer tweets (esto puede necesitar ajustes dependiendo de la estructura de la página)
tweets = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')

for tweet in tweets:
    print(tweet.text)

# Scroll y extraer tweets
for i in range(5):  # Ajusta el rango según la cantidad de scrolls que desees hacer
    # Hacer scroll hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Esperar que carguen nuevos tweets
    time.sleep(SCROLL_PAUSE_TIME)

    # Extraer los tweets visibles en este momento
    tweets = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')

    # Imprimir el texto de los tweets extraídos
    for tweet in tweets:
        print(tweet.text)

# Cerrar el navegador
driver.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="text"]"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00376AB3+25587]
	(No symbol) [0x00309C54]
	(No symbol) [0x00202113]
	(No symbol) [0x00246F62]
	(No symbol) [0x002471AB]
	(No symbol) [0x00287852]
	(No symbol) [0x0026ABE4]
	(No symbol) [0x00285370]
	(No symbol) [0x0026A936]
	(No symbol) [0x0023BA73]
	(No symbol) [0x0023C4CD]
	GetHandleVerifier [0x00654C63+3032483]
	GetHandleVerifier [0x006A6B99+3368153]
	GetHandleVerifier [0x00408F62+624802]
	GetHandleVerifier [0x004107DC+655644]
	(No symbol) [0x0031260D]
	(No symbol) [0x0030F6D8]
	(No symbol) [0x0030F875]
	(No symbol) [0x00301CA6]
	BaseThreadInitThunk [0x767C7BA9+25]
	RtlInitializeExceptionChain [0x778EC11B+107]
	RtlClearBits [0x778EC09F+191]


In [ ]:
def login(username, email, password):
    
    # Navegar a la página de inicio de sesión de Twitter
    driver.get('https://twitter.com/login')
    time.sleep(5)  # Esperar a que cargue la página

    # Buscar el campo del nombre de usuario e ingresar el nombre de usuario o correo
    username_input = driver.find_element(By.NAME, 'text')
    username_input.send_keys(username)
    username_input.send_keys(Keys.RETURN) #Presionar tecla enter
    time.sleep(2)
    
    try:
        # Buscar el campo de confirmación de correo
        email_input = driver.find_element(By.NAME, 'text')
        email_input.send_keys(email)
        email_input.send_keys(Keys.RETURN)
        time.sleep(2)
    except:
        pass
    
    # Buscar el campo de contraseña e ingresar la contraseña
    password_input = driver.find_element(By.NAME, 'password')
    password_input.send_keys(password)
    password_input.send_keys(Keys.RETURN)